# Least Squares using Stochastic Gradient Descent
This notebook contains the code for running multiple models on the ML Higgs boson data using the Least Squares Stochastic Gradient Descent algorithm.

In [18]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from helpers import *
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load train and test data

In [19]:
DATA_TRAIN_PATH = '../data/train.csv'
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [20]:
DATA_TEST_PATH = '../data/test.csv'
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [21]:
X_train.shape

(250000, 30)

## Baseline model using raw data
First let's run the Least Squares SGD algorithm on our raw data without doing any preprocessing. We will use K-fold cross validation to report the metrics on the test data and grid search to tune our hyperparameters.

In [ ]:
param_grid = {
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'num_batches': 10000
}
metrics, params = least_squares_SGD_cv(y_train, X_train, param_grid=param_grid, transform=False)

In [24]:
metrics, params

({'loss': nan, 'accuracy': 0.0, 'f1_score': nan},
 {'max_iters': 1000, 'gamma': 0.01})

Raw data didnt seem to be working with the LS SGD algorithm as expected because we have some outliers and undefined values in our data. Let's preprocess our data and handle missing values.

## Baseline model using lightly feature engineered data
Now let's now preprocess our data a bit to handle the missing values (-999) in various ways.

### All features with NaN values imputed
First let's impute all missing values with median of their respective columns. So we will set the `imputable_th` to `1` which means impute all columns with a nan value ratio less than 1, or in other words all columns.

In [23]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)

In [24]:
tX_train.shape, tX_test.shape

((236483, 31), (568238, 31))

We have now all the columns imputed and plus one more column for the bias.

In [25]:
param_grid = {
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'num_batches': 1000
}
metrics, params = least_squares_SGD_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: overflow encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:96: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: invalid value encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:118: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp / (tp + fn)
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:119: RuntimeWarning: invalid value encountered in long_scalars
  return 2 * precision * recall / (precision + recall)


In [26]:
metrics, params

({'loss': 0.382256474720489,
  'accuracy': 72.55032138024357,
  'f1_score': 0.571939945629682},
 {'max_iters': 1000, 'gamma': 0.01, 'num_batches': 1000})

### All features with NaN values encoded
Now let's instead encode these features with NaN values into new indicator features where the new feature takes on a value of 1 if the value for the feature is missing, otherwise 0.

In [27]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0, encodable_th=1)

In [29]:
tX_train.shape, tX_test.shape

((243430, 31), (568238, 31))

In [30]:
param_grid = {
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'num_batches': 1000
}
metrics, params = least_squares_SGD_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: overflow encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: invalid value encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:96: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient


In [31]:
metrics, params

({'loss': 0.3817942975642752,
  'accuracy': 71.50310150762026,
  'f1_score': 0.5837401342521542},
 {'max_iters': 1000, 'gamma': 0.01, 'num_batches': 1000})

### Mixed imputing and encoding approach
Finally,  let's try a more reasonable approach to the imputing and encoding scheme. As we saw in the exploration notebook, we have some features that have less than 15% of them missing, some around 40% and some more than 70%. Let's impute the columns in the first group, encode the ones in the second group and drop completely the ones in the third group.


In [32]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3, encodable_th=0.7)

In [33]:
tX_train.shape, tX_test.shape

((242240, 24), (568238, 24))

In [34]:
param_grid = {
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'num_batches': 1000
}
metrics, params = least_squares_SGD_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

In [35]:
metrics, params

({'loss': 0.37534499383203207,
  'accuracy': 72.93345442536328,
  'f1_score': 0.5881368654192463},
 {'max_iters': 1000, 'gamma': 0.01, 'num_batches': 1000})

Seems like we get the best performance when we apply a mixed approach. Let's continue our feature engineering with these preprocessing thresholds fixed.

## Baseline model using heavily feature engineered data
In this step, we are going to apply more feature engineering. First, we will apply polynomial features of some degree that we will tune through grid search and cross validation.

In [44]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3, encodable_th=0.7)

In [45]:
tX_train.shape, tX_test.shape

((242240, 24), (568238, 24))

In [46]:
param_grid = {
    'max_iters': 1000,
    'num_batches': 1000,
    'degree': list(range(1, 4)),
    'gamma': [0.01, 0.1],
    'cont_features': [cont_features]
}
metrics, params = least_squares_SGD_cv(ty_train, tX_train, param_grid=param_grid)

/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: overflow encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: invalid value encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:96: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:118: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp / (tp + fn)
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:119: RuntimeWarning: invalid value encountered in long_scalars
  return 2 * precision * recall / (precision + recall)


In [47]:
metrics, params

({'loss': 0.37534499383203207,
  'accuracy': 72.93345442536328,
  'f1_score': 0.5881368654192463},
 {'max_iters': 1000,
  'num_batches': 1000,
  'degree': 1,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20)})

Next, we are going to split our datasets based on the number of jets (`PRI_jet_num`) and create 3 subsets of the data for observations with 0, 1 and more than 1 jet respectively. Each subset will also only have the relevant columns (based on the original paper) All other missing values in the new subsets will be imputed with median values.

In [48]:
X_train_zero, y_train_zero, X_train_one, y_train_one, X_train_many, y_train_many = split_by_jet_num(DATA_TRAIN_PATH, X_train, y_train)
X_test_zero, ids_test_zero, X_test_one, ids_test_one, X_test_many, ids_test_many = split_by_jet_num(DATA_TRAIN_PATH, X_test, ids_test)

In [49]:
X_train_zero.shape, X_train_one.shape, X_train_many.shape

((99913, 15), (77544, 22), (72543, 29))

In [50]:
def train(X_train, y_train, X_test):
    tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3, encodable_th=0.7)
    param_grid = {
        'max_iters': 1000,
        'num_batches': 1000,
        'degree': list(range(1, 4)),
        'gamma': [0.01, 0.1],
        'cont_features': [cont_features]
    }
    metrics, params = least_squares_SGD_cv(ty_train, tX_train, param_grid=param_grid)
    return metrics, params

In [51]:
metrics_zero, params_zero = train(X_train_zero, y_train_zero, X_test_zero)

/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: overflow encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:96: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../implementations.py:67: RuntimeWarning: invalid value encountered in matmul
  return (1 / len(y)) * (tx.T @ ((tx @ w) - y))
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:118: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp / (tp + fn)
/Users/mismayil/Desktop/EPFL/ml-project-1-gradient_surfers/notebooks/../helpers.py:119: RuntimeWarning: invalid value encountered in long_scalars
  return 2 * precision * recall / (precision + recall)


In [52]:
metrics_one, params_one = train(X_train_one, y_train_one, X_test_one)

In [53]:
metrics_many, params_many = train(X_train_many, y_train_many, X_test_many)

In [54]:
metrics_zero, params_zero

({'loss': 0.28620991849413074,
  'accuracy': 79.94654330506297,
  'f1_score': 0.5151338724389386},
 {'max_iters': 1000,
  'num_batches': 1000,
  'degree': 1,
  'gamma': 0.01,
  'cont_features': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)})

In [55]:
metrics_one, params_one

({'loss': 0.398629923964252,
  'accuracy': 69.4254246284501,
  'f1_score': 0.5310136146583313},
 {'max_iters': 1000,
  'num_batches': 1000,
  'degree': 1,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22)})

In [56]:
metrics_many, params_many

({'loss': 0.4160471153978901,
  'accuracy': 69.73604561131836,
  'f1_score': 0.6668531294040978},
 {'max_iters': 1000,
  'num_batches': 1000,
  'degree': 1,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29)})

In [57]:
a = X_train_zero.shape[0]
b = X_train_one.shape[0] 
c = X_train_many.shape[0]
avg_accuracy = ((metrics_zero['accuracy'] * a) +  (metrics_one['accuracy'] * b) + (metrics_many['accuracy'] * c)) / (a + b + c)

print(f"Average accuracy with jet_num training is {avg_accuracy}")

Average accuracy with jet_num training is 73.72034426163664
